# Chapter 8: Building the Complete GPT Model

## From Transformer Blocks to a Full Language Model

In the previous chapters, we built all the individual components:
- **Chapter 4:** Causal Attention (masking future tokens)
- **Chapter 5:** Multi-Head Attention (parallel attention patterns)
- **Chapter 6:** LayerNorm, GELU, FeedForward
- **Chapter 7:** TransformerBlock (combining everything)

Now it's time to **stack these blocks** and add the missing pieces to create a complete GPT model!

```
┌─────────────────────────────────────────────────────────────────────┐
│                         GPT-2 (124M)                                │
│                                                                     │
│  Token IDs ──► Token Embedding ──┐                                  │
│                                  ├──► (+) ──► Dropout               │
│  Positions ──► Pos Embedding ────┘              │                   │
│                                                 ▼                   │
│                                    ┌─────────────────────┐          │
│                                    │ TransformerBlock 1  │          │
│                                    └──────────┬──────────┘          │
│                                               ▼                     │
│                                    ┌─────────────────────┐          │
│                                    │ TransformerBlock 2  │          │
│                                    └──────────┬──────────┘          │
│                                               ▼                     │
│                                             ...                     │
│                                               ▼                     │
│                                    ┌─────────────────────┐          │
│                                    │ TransformerBlock 12 │          │
│                                    └──────────┬──────────┘          │
│                                               ▼                     │
│                                    ┌─────────────────────┐          │
│                                    │     LayerNorm       │          │
│                                    └──────────┬──────────┘          │
│                                               ▼                     │
│                                    ┌─────────────────────┐          │
│                                    │   Output Linear     │──► Logits│
│                                    │   (768 → 50257)     │          │
│                                    └─────────────────────┘          │
└─────────────────────────────────────────────────────────────────────┘
```

## What We'll Build

| Component | Purpose |
|-----------|---------|
| Token Embedding | Convert token IDs → dense vectors |
| Position Embedding | Add position information |
| 12× TransformerBlock | The core processing layers |
| Final LayerNorm | Stabilize before output |
| Output Head | Project to vocabulary logits |

---

## Setup

In [25]:
!pip install torch tiktoken transformers

---

## Part 1: Importing Our Building Blocks

First, we import all the components we built in previous chapters. These are the "Lego bricks" that make up GPT:

| Component | What It Does | Built In |
|-----------|-------------|----------|
| `MultiHeadAttention` | Tokens attend to each other (12 heads in parallel) | Chapter 5 |
| `LayerNorm` | Normalizes activations for stable training | Chapter 6 |
| `GELU` | Smooth activation function | Chapter 6 |
| `FeedForward` | Two-layer MLP (expand 4x, then contract) | Chapter 6 |
| `TransformerBlock` | Combines attention + FFN with residuals | Chapter 7 |

We'll copy these classes here so this notebook is self-contained.

In [26]:
import torch
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, 
                 context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)  
        queries = queries.view(                                             
            b, num_tokens, self.num_heads, self.head_dim                    
        )                                                                   

        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        attn_scores = queries @ keys.transpose(2, 3)
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights @ values).transpose(1, 2)

        context_vec = context_vec.contiguous().view(
            b, num_tokens, self.d_out
        )
        context_vec = self.out_proj(context_vec)
        return context_vec

In [27]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [28]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [29]:
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [30]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate"],
            qkv_bias=cfg["qkv_bias"])
        self.ff = FeedForward(cfg)
        self.norm1 = LayerNorm(cfg["emb_dim"])
        self.norm2 = LayerNorm(cfg["emb_dim"])
        self.drop_shortcut = nn.Dropout(cfg["drop_rate"])

    def forward(self, x):

        shortcut = x
        x = self.norm1(x)
        x = self.att(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        shortcut = x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut
        return x

---

## Part 2: GPT-2 Configuration

Before building the model, we define all the hyperparameters. GPT-2 comes in different sizes:

| Model | Parameters | Layers | Heads | Embedding Dim |
|-------|-----------|--------|-------|---------------|
| GPT-2 Small | **124M** | 12 | 12 | 768 |
| GPT-2 Medium | 355M | 24 | 16 | 1024 |
| GPT-2 Large | 774M | 36 | 20 | 1280 |
| GPT-2 XL | 1.5B | 48 | 25 | 1600 |

We'll implement the **124M** version - small enough to train on a single GPU, but large enough to generate coherent text.

### Configuration Parameters Explained

| Parameter | Value | Meaning |
|-----------|-------|---------|
| `vocab_size` | 50,257 | GPT-2's BPE vocabulary (50k tokens + special tokens) |
| `context_length` | 1,024 | Maximum sequence length the model can process |
| `emb_dim` | 768 | Size of token embeddings (d_model) |
| `n_heads` | 12 | Number of attention heads (768 ÷ 12 = 64 per head) |
| `n_layers` | 12 | Number of TransformerBlocks stacked |
| `drop_rate` | 0.1 | 10% dropout for regularization |
| `qkv_bias` | False | No bias in Q, K, V projections (GPT-2 choice) |

In [31]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

### Quick Test: Verify TransformerBlock Works

Before building the full model, let's verify our TransformerBlock works with the config:

In [32]:
torch.manual_seed(123)
x = torch.rand(2, 4, 768)
block = TransformerBlock(GPT_CONFIG_124M)
output = block(x)

print("Input shape:", x.shape)
print("Output shape:", output.shape)

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768])


---

## Part 3: The Complete GPTModel Class

Now we build the **full GPT model** by combining:

1. **Token Embeddings** - Lookup table: token ID → vector
2. **Position Embeddings** - Lookup table: position → vector  
3. **Embedding Dropout** - Regularization
4. **12 TransformerBlocks** - The core of GPT
5. **Final LayerNorm** - Stabilize outputs
6. **Output Head** - Project to vocabulary logits

### The Forward Pass Visualized

```
        Token IDs: [15496, 11, 314, 716]  ("Hello, I am")
                           │
           ┌───────────────┴───────────────┐
           ▼                               ▼
    ┌─────────────┐                 ┌─────────────┐
    │ tok_emb     │                 │ pos_emb     │
    │ (50257×768) │                 │ (1024×768)  │
    └──────┬──────┘                 └──────┬──────┘
           │                               │
           ▼                               ▼
    (batch, seq, 768)              (seq, 768)
           │                               │
           └───────────────┬───────────────┘
                           │ ADD (+)
                           ▼
                   (batch, seq, 768)
                           │
                    ┌──────▼──────┐
                    │  drop_emb   │  Dropout(0.1)
                    └──────┬──────┘
                           │
              ┌────────────▼────────────┐
              │   12× TransformerBlock   │
              │   (attention + FFN)      │
              └────────────┬────────────┘
                           │
                    ┌──────▼──────┐
                    │ final_norm  │  LayerNorm
                    └──────┬──────┘
                           │
                    ┌──────▼──────┐
                    │  out_head   │  Linear(768 → 50257)
                    └──────┬──────┘
                           │
                           ▼
            Logits: (batch, seq, 50257)
            
            For each position, we get a score for
            every token in the vocabulary!
```

### Understanding the Output

The model outputs **logits** (unnormalized scores) for each vocabulary token:
- Shape: `(batch_size, sequence_length, vocab_size)`
- To get probabilities: `softmax(logits, dim=-1)`
- To get predictions: `argmax(logits, dim=-1)`

In [33]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])

        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)

        pos_embeds = self.pos_emb(
            torch.arange(seq_len, device=in_idx.device)
        )
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits

In [34]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,     # Vocabulary size
    "context_length": 1024,  # Context length
    "emb_dim": 768,          # Embedding dimension
    "n_heads": 12,           # Number of attention heads
    "n_layers": 12,          # Number of layers
    "drop_rate": 0.1,        # Dropout rate
    "qkv_bias": False        # Query-Key-Value bias
}

---

## Part 4: Testing with Real Data

Now let's test our GPT model with actual text! We'll use the dataloader from Chapter 1 to create proper training batches.

### Redefine the Config (for clarity)

### The Data Pipeline for Language Modeling

GPT is trained to predict the **next token**. The dataloader creates input-target pairs:

```
"Frankenstein" text
        │
        ▼ tokenize (BPE)
   [171, 119, 123, 464, 4935, ...]  ◄── Token IDs
        │
        ▼ sliding window
   ┌─────────────────────────────────────┐
   │ Input:  [tok_0, tok_1, ..., tok_255]│
   │ Target: [tok_1, tok_2, ..., tok_256]│  ◄── Shifted by 1!
   └─────────────────────────────────────┘
        │
        ▼ DataLoader batches
   Shape: (batch_size, seq_len) = (2, 256)
        │
        ▼ GPT Model
   Shape: (2, 256, 50257)  ◄── Logits over vocabulary
```

**Key Insight:** The target is always the input shifted by one position. This is how GPT learns to predict "what comes next"!

### Step 1: Download Training Text

In [ ]:
import tiktoken
import requests
from torch.utils.data import Dataset, DataLoader

url = "https://www.gutenberg.org/cache/epub/84/pg84.txt"
filename = "pg84.txt"

try:
    response = requests.get(url)
    response.raise_for_status()
    with open(filename, "wb") as f:
        f.write(response.content)
    print(f"Successfully downloaded '{filename}'")
except requests.exceptions.RequestException as e:
    print(f"Error downloading the file: {e}")

# Load the text
with open("pg84.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

print(f"Text length: {len(raw_text)} characters")

Successfully downloaded 'pg84.txt'
Text length: 438807 characters


### Step 2: Define Dataset and DataLoader

The `GPTDatasetV1` class creates training examples using a **sliding window**:

```
Text: "The cat sat on the mat"
Tokens: [464, 3797, 3332, 319, 262, 2603]

With max_length=4, stride=4 (no overlap):

Example 1:
  Input:  [464, 3797, 3332, 319]   "The cat sat on"
  Target: [3797, 3332, 319, 262]   "cat sat on the"

Example 2:  
  Input:  [262, 2603, ...]          "the mat ..."
  Target: [2603, ...]               "mat ..."
```

**Parameters:**
- `max_length`: How many tokens per training example
- `stride`: How far to slide between examples (stride=max_length means no overlap)

In [ ]:

class GPTDatasetV1(Dataset):
    """Creates input-target pairs using sliding window for next-token prediction."""
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt)

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]


def create_dataloader_v1(txt, batch_size=4, max_length=256,
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):
    """Create a DataLoader with GPT-2 BPE tokenization."""
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

### Step 3: Create DataLoader and Inspect a Batch

Now let's create a dataloader and look at what we get:

**Important requirements for GPT input:**
- Must be **2D tensor**: `(batch_size, sequence_length)`
- Must be **integer token IDs**, not floats
- Sequence length must be ≤ `context_length` (1024)

In [ ]:

# Create dataloader with appropriate settings for GPT-2
# max_length should be <= context_length (1024 for GPT-2)
dataloader = create_dataloader_v1(
    raw_text, 
    batch_size=2,       # Small batch for testing
    max_length=256,     # Sequence length (must be <= 1024)
    stride=256,         # Non-overlapping chunks
    shuffle=False
)

# Get a batch
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

print("Input shape:", inputs.shape)   # (batch_size, seq_len) = (2, 256)
print("Target shape:", targets.shape)
print("\nFirst 10 tokens of input[0]:", inputs[0][:10])
print("First 10 tokens of target[0]:", targets[0][:10])  # Shifted by 1!

Input shape: torch.Size([2, 256])
Target shape: torch.Size([2, 256])

First 10 tokens of input[0]: tensor([  171,   119,   123,   464,  4935, 20336, 46566,   286, 45738,    26])
First 10 tokens of target[0]: tensor([  119,   123,   464,  4935, 20336, 46566,   286, 45738,    26,  1471])


### Step 4: Pass Data Through the GPT Model

Now the exciting part - let's feed our batch through the model!

```
Input:  (2, 256)      ──► GPT Model ──►  Output: (2, 256, 50257)
        ↑                                         ↑
   batch of 2                              logits for each
   sequences,                              position over
   256 tokens each                         50,257 vocab tokens
```

In [ ]:

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)

# Forward pass - inputs are token IDs with shape (batch, seq_len)
with torch.no_grad():
    logits = model(inputs)

print("Input shape:", inputs.shape)      # (2, 256) - token IDs
print("Output shape:", logits.shape)     # (2, 256, 50257) - logits for each position

# The output is logits over the vocabulary for each position
# To get predicted tokens, we take argmax
predicted_tokens = torch.argmax(logits, dim=-1)
print("\nPredicted tokens shape:", predicted_tokens.shape)
print("First 10 predicted tokens:", predicted_tokens[0][:10])

Input shape: torch.Size([2, 256])
Output shape: torch.Size([2, 256, 50257])

Predicted tokens shape: torch.Size([2, 256])
First 10 predicted tokens: tensor([ 7733, 35073, 42775, 44367, 29289,  9580, 39155, 38019, 34764, 23338])


---

## Part 5: Understanding Model Size

Let's count the parameters to understand where the "124M" comes from.

In [24]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total number of parameters: {total_params:,}")

Total number of parameters: 163,009,536


### Total Parameter Count

Let's see exactly how many parameters our model has:

In [40]:
print("Token embedding layer shape:", model.tok_emb.weight.shape)
print("Output layer shape:", model.out_head.weight.shape)

Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])


### Weight Tying: A Clever Trick

Notice something interesting - the token embedding and output head have the **same shape**:

In [41]:
total_params_gpt2 = (
    total_params - sum(p.numel()
    for p in model.out_head.parameters())
)
print(f"Number of trainable parameters "
      f"considering weight tying: {total_params_gpt2:,}"
)

Number of trainable parameters considering weight tying: 124,412,160


In the original GPT-2, these weights are **tied** (shared), saving ~38M parameters!

```
Token Embedding: (50257, 768) = 38.6M parameters
Output Head:     (50257, 768) = 38.6M parameters

Without tying: 38.6M + 38.6M = 77.2M
With tying:    38.6M (shared) = 38.6M  ◄── Saves half!
```

This is called **weight tying** - the intuition is that the embedding of a word and the output prediction for that word should use the same representation.

### Memory Footprint

Each parameter is stored as a 32-bit float (4 bytes). Let's calculate the model size:

In [42]:
total_size_bytes = total_params * 4
total_size_mb = total_size_bytes / (1024 * 1024)
print(f"Total size of the model: {total_size_mb:.2f} MB")

Total size of the model: 621.83 MB


---

## Part 6: Text Generation

Now for the fun part - let's make GPT generate text! Even though it's untrained (random weights), we can still see the generation process in action.

### How Text Generation Works (Autoregressive Decoding)

GPT generates text **one token at a time**, feeding its own output back as input:

```
Step 1: "Hello" ──► GPT ──► predict "," 
Step 2: "Hello," ──► GPT ──► predict " I"
Step 3: "Hello, I" ──► GPT ──► predict " am"
Step 4: "Hello, I am" ──► GPT ──► predict " a"
...and so on

┌─────────────────────────────────────────────────────────┐
│                  Autoregressive Loop                    │
│                                                         │
│   Input: [tok_0, tok_1, ..., tok_n]                    │
│              │                                          │
│              ▼                                          │
│           GPT Model                                     │
│              │                                          │
│              ▼                                          │
│   Logits: (1, n, 50257)                                │
│              │                                          │
│              ▼ take last position only                  │
│   logits[:, -1, :] → (1, 50257)                        │
│              │                                          │
│              ▼ softmax → probabilities                  │
│              ▼ argmax → most likely token               │
│              │                                          │
│   idx_next = predicted token ID                        │
│              │                                          │
│              ▼ append to input                          │
│   Input: [tok_0, tok_1, ..., tok_n, idx_next]          │
│              │                                          │
│              └────── repeat ──────────────────────────►│
└─────────────────────────────────────────────────────────┘
```

### The Simple Generation Function

This function implements **greedy decoding** - always picking the most likely next token:

In [43]:
def generate_text_simple(model, idx,
                         max_new_tokens, context_size): 
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)

        logits = logits[:, -1, :]
        probas = torch.softmax(logits, dim=-1)
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)
        idx = torch.cat((idx, idx_next), dim=1)

    return idx

### Prepare the Starting Prompt

We tokenize a starting prompt and convert it to a tensor. This is the "seed" for generation:

In [44]:
start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)
encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


### Generate New Tokens

Let's generate 6 new tokens starting from "Hello, I am":

**Note:** Since the model is untrained (random weights), the output will be **gibberish**! After training, it would generate coherent text.

In [45]:
model.eval()
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)
print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10


### Decode Back to Text

Convert the token IDs back to human-readable text:

In [46]:
decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

Hello, I am Featureiman Byeswickattribute argue


### Comparing Input, Prediction, and Target

Let's visualize how the untrained model's predictions compare to what we actually want:

In [ ]:


tokenizer = tiktoken.get_encoding("gpt2")

# Decode input and predicted output
input_text = tokenizer.decode(inputs[0].tolist())
predicted_text = tokenizer.decode(predicted_tokens[0].tolist())
target_text = tokenizer.decode(targets[0].tolist())

print("=" * 60)
print("INPUT TEXT (first 200 chars):")
print("=" * 60)
print(input_text[:200])

print("\n" + "=" * 60)
print("MODEL PREDICTION (first 200 chars) - Random, untrained!")
print("=" * 60)
print(predicted_text[:200])

print("\n" + "=" * 60)
print("TARGET TEXT (first 200 chars) - What we want to predict")
print("=" * 60)
print(target_text[:200])

INPUT TEXT (first 200 chars):
﻿The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with alm

MODEL PREDICTION (first 200 chars) - Random, untrained!
 gearnin canyon 349layer screw globalizationCHAR WARRANTنdiff Mastinding council undecided cyclists can arisesRoom triedichoie162 Exec Under Caval342 Canal Pinball darkest diff Pianoquestionolen Verno

TARGET TEXT (first 200 chars) - What we want to predict
��The Project Gutenberg eBook of Frankenstein; Or, The Modern Prometheus
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with al


---

## 📝 Summary

### What We Built

We assembled a complete **GPT-2 (124M)** model from scratch:

```
┌────────────────────────────────────────────────────────────┐
│                      GPTModel                              │
│                                                            │
│  ┌──────────────┐  ┌──────────────┐                       │
│  │  tok_emb     │  │  pos_emb     │  Embeddings           │
│  │ (50257×768)  │  │ (1024×768)   │                       │
│  └──────┬───────┘  └──────┬───────┘                       │
│         └────────┬────────┘                               │
│                  │ (+)                                     │
│                  ▼                                         │
│         ┌────────────────┐                                │
│         │   drop_emb     │  Dropout(0.1)                  │
│         └────────┬───────┘                                │
│                  ▼                                         │
│         ┌────────────────┐                                │
│         │ trf_blocks     │  12× TransformerBlock          │
│         │  (Sequential)  │                                │
│         └────────┬───────┘                                │
│                  ▼                                         │
│         ┌────────────────┐                                │
│         │  final_norm    │  LayerNorm(768)                │
│         └────────┬───────┘                                │
│                  ▼                                         │
│         ┌────────────────┐                                │
│         │   out_head     │  Linear(768 → 50257)           │
│         └────────┬───────┘                                │
│                  ▼                                         │
│            Logits (vocab scores)                          │
└────────────────────────────────────────────────────────────┘
```

### Key Takeaways

1. **GPT Architecture**:
   - Token embeddings + Position embeddings (added together)
   - Stack of TransformerBlocks (12 for GPT-2 Small)
   - Final LayerNorm + Linear output head

2. **Model Size (124M parameters)**:
   - Embeddings: ~39M (token) + ~0.8M (position)
   - TransformerBlocks: ~7M each × 12 = ~85M
   - Output head: ~39M (can be tied with token embedding)

3. **Input/Output**:
   - Input: Token IDs `(batch, seq_len)` - integers!
   - Output: Logits `(batch, seq_len, vocab_size)` - scores for each token

4. **Text Generation**:
   - Autoregressive: generate one token at a time
   - Feed output back as input
   - Untrained = random gibberish

### What's Next?

The model generates **random garbage** because it's untrained. In the next chapters, we'll:
1. **Train** the model on text data
2. **Load pre-trained weights** from OpenAI's GPT-2
3. Implement better generation strategies (temperature, top-k, top-p)

The architecture is complete - now we just need to teach it to speak! 🚀